<a href="https://colab.research.google.com/github/riarahmasp/B_Study-Case_2/blob/main/Kelompok_B_Study_Case_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h2><strong><font color="blue">Study Case 02 - Data Mining dan Business Intelligence:
MixITup Ice Cream</font></strong></h2></center>

## **Kasus**

* **MixITup** adalah perusahaan rintisan yang dimotori para anak muda yang bergerak di bidang usaha Ice Cream, Boba Drinks, dan Desserts (snacks)
* Saat ini **MixITup** memiliki 3 cabang di Jakarta, Depok, dan Tangerang. **MixITup** mendapat respon positif di masyarakat, antrian dine-in dan pesan-antar produk **MixITup** melebihi kapasitas pelayanan.
* Bulan lalu **MixITup** mendapatkan pendanaan seri B sebesar Rp. 33 Milyar,-
* Rencananya **MixITup** ingin "scale" business-nya ke kota lain dan melakukan massive marketing.
* Melalui iming-iming diskon 5% untuk beberapa item di tokonya dan pembagian voucher promo diskon, **MixITup** berhasil mengumpulkan data pelanggan melalui kartu membernya (loyality program).
* Sebagai Data Analyst junior di **MixITup**, kami akan melakukan proses Data Mining dan memberikan informasi/insight dan saran/rekomendasi/strategi ke perusahaan via data.

* Agar memudahkan upper management, kami membuat presentasi dalam format *google slides* yang dapat ditampilkan juga pada rapat direksi untuk memberikan insight dari data, presentasi dapat diakses pada link berikut: https://docs.google.com/presentation/d/1RiiVO9Afj2rwFE_BN09zYEdYchQOVugTrq9GnmHr0P4/edit?usp=sharing 

## **PRE-PROCESSING DATA**

In [ ]:
pip install umap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3565 sha256=b8b4239180fa5d306169b79e473781f91e6e23a18eb65ae4246a8a696321a275
  Stored in directory: /root/.cache/pip/wheels/65/55/85/945cfb3d67373767e4dc3e9629300a926edde52633df4f0efe
Successfully built umap


In [ ]:
#Import Modules yang diperlukan

import warnings; warnings.simplefilter('ignore')
import umap, numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns
from sklearn import cluster, datasets
from sklearn.metrics import silhouette_score as siluet
from sklearn.metrics.cluster import homogeneity_score as purity
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

sns.set(style='ticks', color_codes=True)
random_state=99

In [ ]:
#IMPORT DATA

# Loading Data:
import pandas as pd

try: # Running Locally, yakinkan "file_" berada di folder "data"
    data = pd.read_csv("datanyaaaaaa", low_memory = False, encoding='utf8')
   except: # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taufiksutanto/ADM-Case-01-EDA/master/data/jakarta.csv
    data = pd.read_csv("datanyaaaaaa", low_memory = False, encoding='utf8')
    
data.shape

In [ ]:
#COMBINING DATASETS kalo lebih dari 1

data = pd.concat([data1, data2, data3])

data
data.info()

In [ ]:
#DROPPING VARIABLES

#Drop duplicates
data.shape #(lihat shape)
data.drop_duplicates()
data.shape #recheck shape

#Drop kolom ga penting
data_kolom_bersih = data.drop(['xxx','yyy'],axis=1) #kalo udah yakin pake inplace=True



In [ ]:
#HANDLE MISSING VALUES


data.info() #cek tipe data
data.isna().sum()

#Lihat MV
sns.heatmap(data.isnull(), cbar = False)
plt.title('Heatmap Missing Values')
plt.show

#Ubah data type
data['xxx'] = data['xxx'].astype('category/float64')

#Isi missing value untuk tipe data float64 dengan rata-rata
data_input_mean = data.fillna(data.mean()) #kalo udah yakin pake inplace = True

#Isi MV dengan 0 dsb
data['xxx'] = data['xxx'].fillna('0')

In [ ]:
#HANDLE OUTLIERS

#Cek outliers
#Visualisasi ngeliat outliers

#Catplot
p_coba = sns.catplot(x = 'xxx', y = 'yyy', data = data)
#Boxplot
ax = sns.boxplot(x='xxx', y='yyy', data=data)

#kalo ada,
data_no_outlier = np.abs(data.'yyy' - data.'yyy'.mean())<=(2*data.'yyy'.std())
#mu - 2s < x < mu + 2s
print(data.shape)
data_no_outlier.head()

data_bersih = data[data_no_outlier]
#Perbandingan shape data bersih dengan data yang ada outlier
print(data_bersih.shape, data.shape)



## **EDA**

Yang ingin dijawab:


1.   Kota mana yang cocok untuk dibangun cabang usaha
2.   Marketing apa yang sesuai untuk usaha
3. Promo produk apa yang harus disediakan di hari tertentu 
4. Produk apa yang diminati oleh kalangan tertentu, mungkin berdasarkan kota, jenis kelamin, pendapatan (ini sabi pake clustering)
5. Spending power tiap kota, kota mana yang berpotensi lebih suka jajan dibanding yang lain



In [ ]:
#1&5, pake fungsi count kota
#mungkin bisa diliat ke kota yg dikit cabangnya, rekomendasikan disitu.
#cek spending power (skor pengeluaran) masyarakat kota tsb, kalo tinggi, gas.

#2-4, segregasi konsumen per kota. cek harifav, flavorfav, & promo

## **CLUSTERING ANALYSIS**

In [ ]:
#MENENTUKAN NILAI K OPTIMUM

#Membagi jenis data based on data type,
cat_var = ['aaa','bbb'] #Data kategori
cont_var = ['xxx','yyy'] #Data numerik kontinu

#Ubah data kategorik ke dalam bentuk binary
for col in cat_var:
  dummies = pd.get_dummies(data[col],prefix=col)
  data = pd.concat([data, dummies], axis=1)
  data.drop(col, axis=1, inplace=True)

data.head()


#Scaling variabel kolom numerik kontinu
mms = MinMaxScaler()
mms.fit(data)
data_transformed = mms.transform(data)

#Untuk setiap nilai k, akan diinisialisasi k-means dan inertianya untuk
#identifikasi squared distance sampel ke cluster terdekat

ss_distance = []
K = range(1,15)
for k in K:
  km = KMeans(n_clusters = k)
  km = km.fit(data_transformed)
  ss_distance.append(km.inertia_)

#Seiring nilai k bertambah, nilai sum of squared distance (ss_distance) 
#mendekati nol.

#Plot Elbow Method:
plt.plot(K, ss_distance, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum of Squared Distance')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
#K-MEANS++ CLUSTERING

#Countplot
k=3 #INGAT INPUT K YANG OPTIMUM OK
kmPP = cluster.KMeans(n_clusters=k, init='k-means++', max_iter=300, tol=0.0001, n_jobs=-
1, random_state = random_state)
kmPP.fit(X)
C_kmpp = kmPP.predict(X)
sns.countplot(C_kmpp)
C_kmpp[:10]

#SubPlot
fig, ax = plt.subplots()
ax.scatter(X2D[:,0], X2D[:,1], c=C_kmpp)
plt.show()


In [ ]:
#MINI BATCH K-MEANS++
#Digunakan kalau batch nya kecil

#CountPlot
mbkmPP = cluster.MiniBatchKMeans(n_clusters=k, init='k-means++', max_iter=300,
tol=0.0001, random_state = random_state)
mbkmPP.fit(X)
C_mbkmPP = mbkmPP.predict(X)
sns.countplot(C_mbkmPP)
C_mbkmPP[:10] 

#SubPlot
fig, ax = plt.subplots()
ax.scatter(X2D[:,0], X2D[:,1], c=C_mbkmPP)
plt.show() 